In [20]:
import sys
sys.path.append('../vocaldetection/')
import sklearn
import utils
import json
import os
import pandas as pd
import numpy as np

In [5]:
# Read features and labels
FEAT_PATH = os.environ["FEAT_PATH"]
PIECES = os.environ["PIECES_JSON"]

In [6]:
music_files = []

with open(PIECES) as json_file:  
    data = json.load(json_file)

    for music in data.keys():
        music_files.append(FEAT_PATH+music)

In [73]:
train_features = []
train_labels = []


for tf in music_files:
    # Load VGGish audio embeddings
    #vggish = pd.read_csv(tf+"_VGGish_PCA.csv",index_col=None, header=None)
    #vggish = vggish.values
    
    # Load MFCCs features
    mfccs = pd.read_csv(tf+"_mfcc.csv",index_col=None, header=None)
    mfccs = mfccs.values.T
    
    # Load VocalVariance features
    vv = pd.read_csv(tf+"_vocalvar.csv",index_col=None, header=None)
    vv = vv.values.T

    # Read labels for each frame
    labels = pd.read_csv(tf+"_lbl.csv",index_col=None, header=None)
    labels = labels.values[0]
    
    feature_vector = []
    for idx in range(len(labels)):
        feature_vector.append(np.concatenate((mfccs[idx], vv[idx]), axis=0))
        
        
    # Store the feature vector and corresponding label in integer format
    for idx in range(len(feature_vector)):
        train_features.append(feature_vector[idx])
        train_labels.append(labels[idx])


In [96]:
X = np.array(train_features)
y = np.array(train_labels)

In [97]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [98]:
# Split dataset into train, validation and test sets

In [99]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [100]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [101]:
from sklearn.model_selection import KFold # import KFold
seed = 7
kf = KFold(n_splits=10, random_state=seed) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

10

In [102]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 2704  2705  2706 ... 27036 27037 27038] TEST: [   0    1    2 ... 2701 2702 2703]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [2704 2705 2706 ... 5405 5406 5407]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [5408 5409 5410 ... 8109 8110 8111]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [ 8112  8113  8114 ... 10813 10814 10815]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [10816 10817 10818 ... 13517 13518 13519]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [13520 13521 13522 ... 16221 16222 16223]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [16224 16225 16226 ... 18925 18926 18927]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [18928 18929 18930 ... 21629 21630 21631]
TRAIN: [    0     1     2 ... 27036 27037 27038] TEST: [21632 21633 21634 ... 24333 24334 24335]
TRAIN: [    0     1     2 ... 24333 24334 24335] TEST: [24336 24337 24338 ... 27036 27037 27038]


TypeError: only integer scalar arrays can be converted to a scalar index

In [61]:
# Fit models

# Evaluate models

In [14]:
f0line.shape

(1, 393)

In [ ]:
# Create a scale object
scaler = sklearn.preprocessing.StandardScaler()

# Learn the parameters from the training data only
scaler.fit(train_features)

# Apply the learned parameters to the training and test sets:
train_features_scaled = scaler.transform(train_features)

In [ ]:
test_features_scaled = scaler.transform(test_features)

In [ ]:
# save the scaler to disk
filename = 'scaler_VGGish.sav'
joblib.dump(scaler, filename)

In [ ]:
# Use scikit-learn to train a model with the training features we've extracted

# Lets use a SVC with folowing C parameters: 
params = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]

for c in params:
    clf = sklearn.svm.SVC(C=c)

    # Fit (=train) the model
    clf.fit(train_features_scaled, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_SVM_'+str(c)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf.predict(test_features_scaled)
    
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf.predict(test_features_scaled)

In [ ]:
# Biased Guess Accuracy
ones = np.ones(len(predictions))
accuracy = sklearn.metrics.accuracy_score(test_labels, ones)
print(accuracy)

In [ ]:
# SVM Accuracy
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)

In [ ]:
# lets compute the show the confusion matrix:
cm = sklearn.metrics.confusion_matrix(test_labels, predictions)
print(cm)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(cm, interpolation='nearest', cmap='gray')
for i, line in enumerate(cm):
    for j, l in enumerate(line):
        ax.text(j, i, l, size=20, color='green')
ax.set_xticks(range(len(cm)))
ax.set_xticklabels(labels)
ax.set_yticks(range(len(cm)))
ax.set_yticklabels(labels)
ax.set_ylabel('True label')
ax.set_xlabel('Predicted label')
plt.show()

### Random Forest Classifier without scaler

In [ ]:
params = [100, 200, 500, 1000]

for estimators in params:
    clf3 = RandomForestClassifier(n_estimators=estimators)

    # Fit (=train) the model
    clf3.fit(train_features, train_labels)
    
    # save the model to disk
    filename = 'finalized_model_RF_'+str(estimators)+'_VGGish.sav'
    print (filename)
    joblib.dump(clf3, filename)
    
    # Now lets predict the labels of the test data!
    predictions = clf3.predict(test_features)
    # We can use sklearn to compute the accuracy score
    accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
    print(accuracy)

In [ ]:
# Now lets predict the labels of the test data!
predictions = clf3.predict(test_features)

In [ ]:
# We can use sklearn to compute the accuracy score
accuracy = sklearn.metrics.accuracy_score(test_labels, predictions)
print(accuracy)